# Log-normal people

In [5]:
import transformers
import shap
import numpy
import matplotlib.pylab as plt
import pandas as pd
import numpy as np
import scipy
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Opis rozwiązania
Celem projektu jest Nasze rozwiązanie składa się z 3 modułów. Dwa z nich to modele, które pozwalają klasyfikować tekst. Pierwszy przewiduje sentyment, a drugi rodzaj negatywnego nacechowania. A trzeci służy do objaśniania predykcji modelu. Można je łączyć na kilka sposobów, które znajdą zastosowanie w wielu produktach takich jak:
1. Analiza sentymentu po stronie użytkownika. Wpisując komentarz otrzymujemy informację czy nie jest on nacechowany negatywnie (toksyczność, groźba, dyskryminacja grup). Jeżeli jest, to nasz moduł objaśniania wskaże dla każdej z negatywnych cech miejsce jej występowania w komentarzu, które **użytkownik będzie mógł poprawić**.

![](./img/product1.png)

2. Automatyczne zgłaszanie negatywnych komentarzy. Po wykryciu negatywnego sentymentu moderator otrzymuje komentarz z oznaczonym negatywnym fragmentem, co przyspiesza i ułatwia moderację. Szczególnie w przypadku długich tekstów, ponieważ **natychmiast wyróżnione są interesujące fragmenty**. W naszym rozwiązaniu chcemy, aby ostateczną decyzję zawsze podejmował moderator, nasz model ma mu wyłącznie ułatwić pracę. Żadne z dostostępnych na rynku narzędzi nie pozwala na tak dokładne wróżnienie wybranej negatywnej klasy. OpenAI pozwala na oznaczenie pozytywny/negatywny, podobnie wygląda to na Monkeylearn. Możemy "tłumaczyć" wybór dowolnej z negatywnych klas, nawet po zmianie modelu np. na taki który posiada więcej klas negatywnych. 

![](./img/product2.png)

Ponadto to rozwiązanie można rozszerzyć np. o inne media takie jak video czy audio, dla których tekst możemy otrzymać z pomocą transkrypcji video.

# Moduł objaśniania modeli
Jest to nasz najważniejszy moduł, korzystając z biblioteki SHAP można objaśniać dowolne modele, także te tekstowe. Poniżej znajduje się przykład z biblioteki shap, w kolejnych rozdziałach pokażemy zastosowanie tego modułu we współpracy z naszymi modelami.
![](./img/shap.png)

Tutaj dla każdego fragmentu tekstu, oznaczone mamy jaki miał wpływ na wyjście modelu. Dzięki temu jesteśmy w stanie dla wybranego negatywnego zachowania określić miejsce jego występowania w tekście.

## Moduł badania sentymentu
Moduł ten odpowiada za badanie sentymenu, sklasyfikuje dowolny tekst do jednej z 3 klas: 
- Negative
- Neutral
- Positive


In [6]:
sentiment_pipe = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment")

def translate(label):
    t = {"LABEL_0": "Negative",
     "LABEL_1": "Neutral",
     "LABEL_2": "Positive"}
    return t[label]

In [7]:
def predict_sentiment(data):
    label = translate(sentiment_pipe(data)[0]["label"])
    return {"label": label, "score": sentiment_pipe(data)[0]["score"]}

### Przykład użycia

In [8]:
text = "you dumb clown"
predict_sentiment(text)

{'label': 'Negative', 'score': 0.9534616470336914}

Mając negatywny wynik możemy z pomocą `Modułu objaśniania modeli` znaleźć co sprawiło, że model oznaczył ten tekst jako `Negative`.

In [12]:
explainer = shap.Explainer(sentiment_pipe) # Moduł objaśniania
shap_values = explainer([text])
label = sentiment_pipe(text)[0]["label"]
shap.plots.text(shap_values[:,:, label]) # Wybieramy klasę objaśnianą (klasa )

W skali od niebieskiego do czerwonego mamy pokazane, które fragmenty tekstu zadecyowały o wyniku modelu. Tutaj słowa `dumb` oraz `clown` zadecydowały o  klasie `Negative`.
Jeżeli zmienimy tekst, sprawimy, że będzie bardziej pozytywny.

In [13]:
text = "you dumb clown, but I still like you"

In [15]:
print(predict_sentiment(text))
shap_values = explainer([text])
label = sentiment_pipe(text)[0]["label"]
shap.plots.text(shap_values[:,:, label]) # Wybieramy klasę objaśnianą

{'label': 'Negative', 'score': 0.5696905851364136}


Sentyment nadal jest `Negative`, ale widzimy, że moduł objaśniający pokazuje nam, które słowa zmniejszyły prawdopodobieństwo tej klasy.

# Moduł klasyfikacji negatywnego tekstu
Dla dowolnego tekstu oprócz tego, że tekst jest negatywny możemy określić dlaczego jest negatywny tj. wskazać prawdopodobieństwo należenia do każdej z klas:
- toxic
- severe_toxic
- obscene
- threat
- insult
- identity_hate

Wracając do poprzedniego przykładu, obliczymy te prawdopodobieństwa.

In [16]:
# Tworzenie modelu klasyfikacji negatywnego tekstu
clf_tokenizer = AutoTokenizer.from_pretrained("unitary/toxic-bert")
clf_model = AutoModelForSequenceClassification.from_pretrained("unitary/toxic-bert")
clf_pred = transformers.pipeline("text-classification", model=clf_model, tokenizer=clf_tokenizer, return_all_scores=True)

In [17]:
text = "you dumb clown" # Przykład
clf_pred(text)

[[{'label': 'toxic', 'score': 0.7617954015731812},
  {'label': 'severe_toxic', 'score': 0.00035054481122642756},
  {'label': 'obscene', 'score': 0.014293862506747246},
  {'label': 'threat', 'score': 1.986711322388146e-05},
  {'label': 'insult', 'score': 0.22305439412593842},
  {'label': 'identity_hate', 'score': 0.0004858026804868132}]]

Najbardziej prawdopodobne jest, że tekst ten zawiera klasy: `toxic` oraz `insult`. Poniżej objaśniłem dlaczgo model z dużym prawdopodobieństwem przypisał klasę `toxic`:

In [18]:
clf_explainer = shap.Explainer(clf_pred)
shapley_values = clf_explainer(np.array([text]))
shap.plots.text(shapley_values[:, :, 'toxic']) # Klasa toxic

Oraz klasę `insult`:

In [19]:
shap.plots.text(shapley_values[:, :, 'insult']) # Klasa insult

# Przykłady 
## Pozytywny tekst
Jak dotąd wszystkie zaprezentowane przykłady były negatywne, poniżej zaprezentowałem jak model reaguje na pozytywne komentarze. 

In [20]:
text = "I really enjoyed your work, keep it up"
print(predict_sentiment(text))

{'label': 'Positive', 'score': 0.9836941957473755}


In [21]:
clf_pred(text)

[[{'label': 'toxic', 'score': 0.47984838485717773},
  {'label': 'severe_toxic', 'score': 0.0829383134841919},
  {'label': 'obscene', 'score': 0.12115631252527237},
  {'label': 'threat', 'score': 0.1012764424085617},
  {'label': 'insult', 'score': 0.11856300383806229},
  {'label': 'identity_hate', 'score': 0.09621746838092804}]]

## Długi tekst

In [22]:
text = """Have you heard the term “you get what you pay for”? All your troubles can be resolved, 
simply by purchasing a business class ticket or better yet a first class seat and forever quit 
your complaining. Maybe use all those air miles you’ve racked up to upgrade from economy next 
time. You lack a ton of common sense and have publicly embarrassed yourself with these whiny comments."""

In [23]:
predict_sentiment(text)

{'label': 'Negative', 'score': 0.868039608001709}

In [24]:
clf_pred(text)

[[{'label': 'toxic', 'score': 0.9142789840698242},
  {'label': 'severe_toxic', 'score': 0.0007636288064531982},
  {'label': 'obscene', 'score': 0.008510921150445938},
  {'label': 'threat', 'score': 0.0014207024360075593},
  {'label': 'insult', 'score': 0.07197616994380951},
  {'label': 'identity_hate', 'score': 0.0030496353283524513}]]

In [31]:
shapley_values = clf_explainer(np.array([text]))
shap.plots.text(shapley_values[:, :, 'insult']) # Klasa insult

Model działa również dla długich tekstów, poprawnie rozpoznał iż komentaż był negatywnie nastawiony oraz toksyczny. Ponadto dokładnie wskazał mniejsce, w którym autor artykułu został urażony.

## Komentarze z Reddita

In [32]:
text = """
We should tax the s**t out of soft drinks and other fast foods and snacks. Sodium and sugar, coupled with a sedentary lifestyle are the true killers.
How ironic that obesity is one of the leading comorbidities associated with having a bad run at nearly every kind of illness.
"""

In [33]:
predict_sentiment(text)

{'label': 'Negative', 'score': 0.9535958766937256}

In [34]:
clf_pred(text)

[[{'label': 'toxic', 'score': 0.7127734422683716},
  {'label': 'severe_toxic', 'score': 0.0027107808273285627},
  {'label': 'obscene', 'score': 0.19764475524425507},
  {'label': 'threat', 'score': 0.0018170236144214869},
  {'label': 'insult', 'score': 0.07720550149679184},
  {'label': 'identity_hate', 'score': 0.00784845370799303}]]

Przedstawię wykres dla klasy `obscene`, ponieważ dla klasy `toxic` prawdopodobieństwo jest relatywnie niskie.

In [36]:
shapley_values = clf_explainer(np.array([text]))
shap.plots.text(shapley_values[:, :, 'obscene']) # Klasa obscene